In [ ]:
from transformers import (
    MBartForConditionalGeneration, MBartTokenizer, 
    Seq2SeqTrainingArguments, Seq2SeqTrainer
  )

import torch
from torch.utils.data import random_split

In [ ]:
import pandas as pd
data = pd.read_csv('../input/english-to-hindi-parallel-dataset/newdata.csv')
data.head(2)

In [ ]:
data.shape

In [ ]:
file = []
for  index, row in data.iterrows():
      file.append(
          {
              "translation": {
                  "hi": row['hindi_sentence'],
                  "en": row['english_sentence']
              }
          }
      )
print(f'total size of data is {len(file)}')

In [ ]:
def data_collator(features:list):

  labels = [f["translation"]["en"] for f in features]
  inputs = [f["translation"]["hi"] for f in features]

  batch = tokenizer.prepare_seq2seq_batch(src_texts=inputs, src_lang="hi_IN", tgt_lang="en_XX", tgt_texts=labels, max_length=32, max_target_length=32)

  for k in batch:
    batch[k] = torch.tensor(batch[k])

  return batch

In [ ]:
split = 0.98
train_dataset, eval_dataset = random_split(file, lengths=[int((1-split)*len(file))+1, int(split*len(file))])

In [ ]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-cc25")
tokenizer = MBartTokenizer.from_pretrained("facebook/mbart-large-cc25")

In [ ]:
# defining training related arguments
args = Seq2SeqTrainingArguments(output_dir="indic-mbart",
                        do_train=True,
                        do_eval=True,
                        evaluation_strategy="epoch",
                        per_device_train_batch_size=16,
                        per_device_eval_batch_size=16,
                        learning_rate=5e-5,
                        num_train_epochs=1,
                        logging_dir="/logs")

In [ ]:
trainer = Seq2SeqTrainer(model=model, 
                args=args, 
                data_collator=data_collator, 
                train_dataset=train_dataset,
                eval_dataset=eval_dataset)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("mbart-hin-eng")

In [ ]:
!ls  mbart-hin-eng

In [ ]:
model = MBartForConditionalGeneration.from_pretrained('mbart-hin-eng')

In [ ]:
T1 = "अंतिम प्रविष्ट घटना को हाइलाइट करो"
inputs = tokenizer(T1, return_tensors="pt")
inputs

In [ ]:
device = "cuda:0"
model = model.to(device)
inputs = inputs.to(device)

In [ ]:
# T1 = "अंतिम प्रविष्ट घटना को हाइलाइट करो"
# inputs = tokenizer(T1, return_tensors="pt")
# # translated_tokens = model(inputs)
translated_tokens = model.generate(**inputs,forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])


In [ ]:
translated_tokens

In [ ]:
tokenizer.batch_decode(translated_tokens)[0]